## horseIDとjockeyIDの取得

In [61]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
from tqdm import tqdm

pd.options.display.max_rows = 5
pd.options.display.max_columns = 100
pd.options.display.max_colwidth = 10

In [62]:

df = pd.read_pickle('inner_df.pickle')
race_id_list = df.index.unique()
type(race_id_list[0])
# race_id_list = ["202101010101","202101010102"]

str

In [63]:
test = {}
def scrape_race_results(race_id_lists,par_race_results={}):
  race_results = par_race_results
  for _race_id in tqdm(race_id_lists):
    # if _race_id in race_result.keys():
    #   continue
    time.sleep(1)
    try:
      # スクレイピング
      url = "https://race.netkeiba.com/race/result.html?race_id=" + _race_id
      html = requests.get(url)
      html.encoding= "EUC_JP"
      soup = BeautifulSoup(html.text, "html.parser")
      
      # horse_id
      horse_id_list = []      
      horse_a_list = soup.find('table',attrs={"summary":"全着順"}).find_all('a',attrs={"href":re.compile("/horse/")})
      for a in horse_a_list:
        horse_id = re.findall(r'\d+',a["href"])
        horse_id_list.append(horse_id[0])
      Series1 = pd.Series(data = horse_id_list,dtype=str,name="horse_id")
      # jockey_id
      jockey_id_list = []
      jockey_a_list = soup.find('table',attrs={"summary":"全着順"}).find_all('a',attrs={"href":re.compile("/jockey/")})
      for u in jockey_a_list:
        jockey_id = re.findall(r'\d+',u["href"])
        jockey_id_list.append(jockey_id[0])
      Series2 = pd.Series(data = jockey_id_list,dtype=str,name="jockey_id")
      
      results = pd.concat([Series1, Series2], axis=1)
      test[_race_id] = results
      
    except UnicodeDecodeError as e:
      # print(e)
      # print(f"レースID: {race_id} のデータはありません")
      continue
    except KeyboardInterrupt:
      break
    except TimeoutError:
      break
  return test

In [64]:
results_new = scrape_race_results(race_id_list)
results_new

100%|██████████| 24/24 [00:32<00:00,  1.33s/it]


{'202102010101':      horse_id jockey_id
 0   201910...     01182
 1   201910...     01093
 ..        ...       ...
 8   201910...     01177
 9   201910...     01112
 
 [10 rows x 2 columns],
 '202102010102':      horse_id jockey_id
 0   201810...     01030
 1   201810...     01181
 ..        ...       ...
 10  201810...     01144
 11  201810...     01015
 
 [12 rows x 2 columns],
 '202102010103':      horse_id jockey_id
 0   201810...     01095
 1   201810...     01182
 ..        ...       ...
 10  201810...     01185
 11  201810...     01170
 
 [12 rows x 2 columns],
 '202102010104':      horse_id jockey_id
 0   201810...     01144
 1   201810...     01185
 ..        ...       ...
 10  201810...     01182
 11  201810...     01181
 
 [12 rows x 2 columns],
 '202102010105':      horse_id jockey_id
 0   201910...     01156
 1   201910...     05339
 ..        ...       ...
 8   201910...     01181
 9   201910...     01161
 
 [10 rows x 2 columns],
 '202102010106':      horse_id jockey_id

In [65]:
for key in results_new.keys():
  results_new[key].index = [key] * len(results_new[key])
results = pd.concat([results_new[key] for key in results_new.keys()],sort=False)

In [66]:
print(type(results))
print(len(results))
print(type(df))
print(len(df))

<class 'pandas.core.frame.DataFrame'>
313
<class 'pandas.core.frame.DataFrame'>
313


In [67]:
# pd.set_option('display.max_rows',None)

df_add_id = pd.concat([df,results],axis=1)

In [68]:
df_add_id

,着順,枠,馬番,馬名,性齢,斤量,騎手,タイム,着差,人気,単勝オッズ,後3F,コーナー通過順,厩舎,馬体重(増減),date,race_type,course_ren,weather,ground_state,horse_id,jockey_id
202102010101,1,4,4,ナムラリコリス,牝2,52.0,△泉谷,1:09.3,NaN,1,1.4,35.7,2-2,栗東大橋,466(+2),2021年7月3日,芝,1200,晴,良,201910...,01182
202102010101,2,1,1,プラソン,牝2,54.0,藤岡佑,1:09.4,1/2,2,5.5,35.9,1-1,栗東寺島,430(+2),2021年7月3日,芝,1200,晴,良,201910...,01093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202103010112,15,7,14,トランザクト,牡3,52.0,◇藤田菜,1:10.6,1.3/4,5,14.8,39.3,4-7,美浦武藤,496(-4),2021年7月3日,ダ,1150,曇,稍,201810...,01164
202103010112,16,4,8,リゼ,牝5,55.0,石川,1:12.4,大,14,107.5,39.5,15-15,美浦田中清,446(-4),2021年7月3日,ダ,1150,曇,稍,201610...,01150


In [69]:
df_add_id.to_pickle("df_add_id.pickle")

----

## idのリストを作成

In [70]:
horse_id_list = df_add_id["horse_id"].unique()
len(horse_id_list)
# horse_id_list = horse_id_list
# horse_id_list = ['2018105875', '2018105258', '2018101134', '2018102791']

313

In [71]:
# url = "https://db.netkeiba.com/horse/2019109094"
# url = "https://db.netkeiba.com/horse/2018102444"
# time.sleep(1)
# pd.read_html(url,match='日付')

In [72]:
test = {}
def scrape_horse_results(horse_id_list,par_horse_results={}):
  horse_results = par_horse_results
  for _horse_id in tqdm(horse_id_list):
    # if _horse_id in horse_id_list.keys():
    #   continue
    time.sleep(2)
    try:
      # スクレイピング
      url = "https://db.netkeiba.com/horse/" + _horse_id
      time.sleep(1)
      # [0]をつけてリストの中だけを取得
      horse_results[_horse_id] = pd.read_html(url,match='日付')[0]
      
    except UnicodeDecodeError as e:
      # print(e)
      # print(f"レースID: {race_id} のデータはありません")
      continue
    except KeyboardInterrupt:
      break
        
    except TimeoutError:
      break
    # print(horse_results)
      
  return horse_results

In [73]:
horse_results = scrape_horse_results(horse_id_list)

100%|██████████| 313/313 [17:53<00:00,  3.43s/it]


In [74]:
horse_results

{'2019102534':            日付    開催 天気   R       レース名  映像  頭数  枠番  馬番    オッズ  人気  着順    騎手  \
 0   2022/0...  2中京3  曇  11  中スポ賞ファ... NaN  18   4   7  141.9  16  18  泉谷楓真   
 1   2022/0...  1中京5  晴  10     紅梅S(L) NaN   9   4   4   15.2   5   6  泉谷楓真   
 ..        ...   ... ..  ..        ...  ..  ..  ..  ..    ...  ..  ..   ...   
 4   2021/0...  1函館1  晴   1      2歳未勝利 NaN  10   4   4    1.4   1   1  泉谷楓真   
 5   2021/0...  1札幌3  曇   5       2歳新馬 NaN  12   8  11   15.7   8   2  泉谷楓真   
 
     斤量     距離 馬場 馬場指数     タイム   着差 ﾀｲﾑ指数   通過        ペース    上り       馬体重  \
 0   55  芝1400  稍   **  1:23.7  2.8    **  2-2  33.1-36.5  39.1  462(-12)   
 1   55  芝1400  良   **  1:22.5  0.7    **  3-2  34.9-35.3  35.8   474(-2)   
 ..  ..    ... ..  ...     ...  ...   ...  ...        ...   ...       ...   
 4   52  芝1200  良   **  1:09.3 -0.1    **  2-2  33.5-35.8  35.7   466(+2)   
 5   52  芝1200  良   **  1:08.5  0.6    **  4-2  33.6-34.3  34.3    464(0)   
 
     厩舎ｺﾒﾝﾄ  備考  勝ち馬(2着馬)     賞金  
 0      NaN

In [75]:
for key in horse_results.keys():
  horse_results[key].index = [key] * len(horse_results[key])
results = pd.concat([horse_results[key] for key in horse_results.keys()],sort=False)

In [76]:
# pd.set_option('display.max_rows',10)
# results

In [77]:
results.to_csv("horse_vs_data.csv")

In [78]:
df = pd.read_csv('horse_vs_data.csv')
df


/var/folders/g6/k526pb596zv_zcdn87rsfwqc0000gn/T/ipykernel_40322/2339498312.py:1: DtypeWarning: Columns (10,11,29,31,32,33,34,36,37,38,40,43,102,106,107,112,113,114,116,123,124,126,127,128,129,131,132,133,135,138,183,185) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('horse_vs_data.csv')


,Unnamed: 0,日付,開催,天気,R,レース名,映像,頭数,枠番,馬番,オッズ,人気,着順,騎手,斤量,距離,馬場,馬場指数,タイム,着差,ﾀｲﾑ指数,通過,ペース,上り,馬体重,厩舎ｺﾒﾝﾄ,備考,勝ち馬(2着馬),賞金,騎手金,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,...,"('Unnamed: 40_level_0', 'Unnamed: 40_level_1', 'Unnamed: 40_level_2', 'Unnamed: 40_level_3')","('アデウス', 'Unnamed: 41_level_1', 'Unnamed: 41_level_2', 'Unnamed: 41_level_3')","('Unnamed: 42_level_0', 'Unnamed: 42_level_1', 'Unnamed: 42_level_2', 'Unnamed: 42_level_3')",Unnamed: 2,Unnamed: 7,着順着馬),"('日付', '2022/02/06', '2022/01/30', '2021/12/11', '2021/11/07', '2021/09/11', '2021/07/03', '2021/04/10', '2021/02/06', '2021/01/17', '2021/01/05', '2020/10/03', '2020/09/20', '2020/09/13', '2020/08/29', '2020/08/16', '2020/06/06')","('開催', '1小倉8', '1小倉6', '6中京3', '5東京2', '4中山1', '1福島1', '1新潟1', '1東京3', '1中山6', '1中山1', '4中山8', '4中山4', '4中山2', '2札幌5', '3新潟2', '3東京1')","('天気', '晴', '曇', '晴', '晴', '晴', '曇', '晴', '晴', '曇', '晴', '晴', '曇', '曇', '曇', '曇', '曇')","('R', '8', '7', '7', '7', '12', '12', '3', '2', '1', '1', '3', '1', '1', '2', '3', '5')","('レース名', '4歳以上1勝クラス', '4歳以上1勝クラス', '3歳以上1勝クラス', '3歳以上1勝クラス', '3歳以上1勝クラス', '3歳以上1勝クラス', '3歳未勝利', '3歳未勝利', '3歳未勝利', '3歳未勝利', '2歳未勝利', '2歳未勝利', '2歳未勝利', '2歳未勝利', '2歳未勝利', '2歳新馬')","('映像', 'Unnamed: 5_level_1', 'Unnamed: 5_level_2', 'Unnamed: 5_level_3', 'Unnamed: 5_level_4', 'Unnamed: 5_level_5', 'Unnamed: 5_level_6', 'Unnamed: 5_level_7', 'Unnamed: 5_level_8', 'Unnamed: 5_level_9', 'Unnamed: 5_level_10', 'Unnamed: 5_level_11', 'Unnamed: 5_level_12', 'Unnamed: 5_level_13', 'Unnamed: 5_level_14', 'Unnamed: 5_level_15', 'Unnamed: 5_level_16')","('頭数', '11', '16', '16', '16', '16', '16', '15', '16', '16', '16', '16', '13', '12', '12', '18', '10')","('枠番', '8', '6', '3', '1', '7', '6', '8', '8', '1', '8', '8', '2', '7', '2', '7', '2')","('馬番', '11', '12', '5', '1', '13', '11', '15', '16', '2', '16', '15', '2', '9', '2', '15', '2')","('オッズ', '107.9', '53.2', '101.9', '192.4', '14.7', '27.0', '8.4', '6.0', '3.6', '9.0', '18.4', '3.8', '7.7', '92.3', '46.5', '43.9')","('人気', '10', '11', '13', '14', '5', '6', '6', '3', '2', '4', '7', '2', '4', '9', '10', '7')","('着順', '8', '10', '9', '10', '11', '8', '1', '7', '2', '5', '9', '4', '2', '4', '13', '10')","('騎手', '松本大輝', '松本大輝', '亀田温心', '石川裕紀', '藤田菜七', '木幡巧也', '津村明秀', '大野拓弥', '木幡巧也', '木幡巧也', '木幡巧也', '木幡巧也', '木幡巧也', '松田大作', '木幡巧也', '木幡巧也')","('斤量', '53', '53', '55', '56', '53', '54', '56', '56', '56', '56', '55', '54', '54', '54', '54', '54')","('距離', 'ダ2400', 'ダ1700', 'ダ1400', 'ダ1400', 'ダ1200', 'ダ1150', 'ダ1200', 'ダ1300', 'ダ1200', 'ダ1200', '芝1200', 'ダ1200', 'ダ1200', '芝1200', '芝1200', '芝1600')","('馬場', '良', '良', '良', '良', '良', '稍', '良', '良', '良', '良', '良', '良', '稍', '良', '稍', '良')","('馬場指数', '**', '**', '**', '**', '**', '**', '**', '**', '**', '**', '**', '**', '**', '**', '**', '**')","('タイム', '2:39.4', '1:48.5', '1:25.9', '1:26.9', '1:12.5', '1:09.5', '1:12.4', '1:20.5', '1:13.2', '1:13.0', '1:11.0', '1:13.8', '1:13.7', '1:11.2', '1:11.5', '1:39.2')","('着差', '3.3', '2.6', '1.1', '1.6', '1.1', '1.3', '-0.6', '1.5', '0.4', '0.5', '0.3', '0.9', '0.3', '0.9', '1.3', '2.0')","('ﾀｲﾑ指数', '**', '**', '**', '6', '**', '**', '**', '**', '**', '**', '**', '**', '**', '**', '**', '**')","('8', '5-5-8-8', '3-4-4-5', '7-7', '3歳以上1勝クラス', '4-4', '4-4', '2-2', '4-5', '5-3', '3-2', '2-3', '1-1', '3-3', '3-3', '9-9', '9-8')","('4', '38.1-38.3', '30.0-38.6', '34.6-38.3', 'Unnamed: 21_level_4', '33.9-37.5', '31.0-37.2', '34.8-37.6', '29.2-37.7', '34.4-38.4', '34.4-38.1', '35.2-35.5', '34.4-38.5', '34.4-39.0', '34.1-36.2', '34.4-35.8', '37.9-34.3')","('4', '40.7', '40.8', '38.6', '14', '38.4', '38.2', '37.5', '38.6', '38.2', '38.4', '35.7', '39.4', '39.1', '36.9', '36.2', '35.1')","('75.9', '506(+4)', '502(0)', '502(-16)', '1', '510(+2)', '508(+2)', '506(-6)', '512(-2)', '514(-2)', '516(+14)', '502(+4)', '498(-4)', '502(-2